In [6]:
import cv2
import numpy as np

def imgresize(img):
    return cv2.resize(img , (500,500))

def showimage(img):
    cv2.imshow('abhi', img) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def FaceDetectionUsingDNN(imgpath):
    
    modelFile =  "Path for res10_300x300_ssd_iter_140000.caffemodel" #example- path can be witten like this "E:\\res10_300x300_ssd_iter_140000.caffemodel"
    configFile = "path for deploy.prototxt.txt"
    
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

    inp = cv2.imread(imgpath)
    frame = inp
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    #[,frame,number of detection,[classid,class score,conf,x,y,h,w]]
    # loop over the detections
    
    for i in range(0, detections.shape[2]):
        
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.5:
            
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # draw the bounding box of the face along with the associated
            # probability
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(inp, (startX, startY), (endX, endY),(0, 0, 255), 2)

    # show the output image
    inp = imgresize(inp)
    showimage(inp)

In [ ]:
FaceDetectionUsingDNN("InputImagePath")

In [ ]:
#For Realtime Face Detection On Webcam

import cv2
import numpy as np
import imutils

modelFile = "path for res10_300x300_ssd_iter_140000.caffemodel"
configFile = "path for deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    frame = imutils.resize(frame, width=750)

    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))


    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        if confidence < 0.5:
            continue

        # compute the (x, y)-coordinates of the bounding box for the
        # object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # draw the bounding box of the face along with the associated
        # probability
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),
            (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
video_capture.release()